In [2]:
import sklearn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

from gensim.models import Word2Vec

from sklearn import neighbors
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
import re
import string

In [ ]:
#load data
newsgroups_train = fetch_20newsgroups(subset='train',shuffle=True, remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test',shuffle=True, remove=('headers', 'footers', 'quotes'))

In [ ]:
def filter_data():
    #filter data
    # newsgroups_train = fetch_20newsgroups(subset='train',remove=('headers', 'footers', 'quotes'))
    # newsgroups_test = fetch_20newsgroups(subset='test',remove=('headers', 'footers', 'quotes'))
    
    class_names = newsgroups_train.target_names
    print('Training data size::', len(newsgroups_train['data']))
    
    # Printing all the categories
    print('Training categories names::', newsgroups_train.target_names)
    
    # Finding frequency of each category
    targets, frequency = np.unique(newsgroups_train.target, return_counts=True)
    targets_str = np.array(newsgroups_train.target_names)
    
    #Prinnting all frequency
    #print("Print targets, frequency in array::", targets, frequency)
    print(list(zip(targets_str, frequency)))
    
    #Training data class distribution
    #fig=plt.figure(figsize=(10, 5), dpi= 80, facecolor='w', edgecolor='k')
    #plt.bar(targets_str,frequency, color='#ff6961')
    #plt.xticks(rotation=90)
    #plt.title('Class distribution of 20 Newsgroups Training Data')
    #plt.xlabel('News Group')
    #plt.ylabel('Frequency')
    #plt.show()
    
    # Preparing train data
    newsgroups_train_df = pd.DataFrame({'data': newsgroups_train.data, 'target': newsgroups_train.target})
    #print(newsgroups_train_df.head())
    # Preparing test data
    newsgroups_test_df = pd.DataFrame({'data': newsgroups_test.data, 'target': newsgroups_test.target})
    print(newsgroups_test_df)
    
    # Text preprocessing steps - remove numbers, captial letters and punctuation
    newsgroups_train_df.data.str.replace('[^a-zA-Z]', '')
    newsgroups_test_df.data.str.replace('[^a-zA-Z]', '')
    newsgroups_train_df = newsgroups_train_df.data.str.lower()
    newsgroups_test_df = newsgroups_test_df.data.str.lower()
    print(newsgroups_train_df.head())
    

    return newsgroups_train, newsgroups_test, class_names

In [ ]:
def bow_features(train_data, test_data):
    # Bag-of-words representation
    bow_vectorize = CountVectorizer()
    bow_train = bow_vectorize.fit_transform(train_data.data) #bag-of-word features for training data
    bow_test = bow_vectorize.transform(test_data.data)
    feature_names = bow_vectorize.get_feature_names() #converts feature index to the word it represents.
    shape = bow_train.shape
    print('{} train data points.'.format(shape[0]))
    print('{} feature dimension.'.format(shape[1]))
    print('Most common word in training set is "{}"'.format(feature_names[bow_train.sum(axis=0).argmax()]))
    return bow_train, bow_test, feature_names

In [ ]:
def tf_idf_features(train_data, test_data):
    # Bag-of-words representation
    tf_idf_vectorize = TfidfVectorizer(ngram_range=(1, 3), stop_words='english')
    tf_idf_train = tf_idf_vectorize.fit_transform(train_data.data) #bag-of-word features for training data
    feature_names = tf_idf_vectorize.get_feature_names() #converts feature index to the word it represents.
    tf_idf_test = tf_idf_vectorize.transform(test_data.data)
    print(tf_idf_train)
    return tf_idf_train, tf_idf_test, feature_names

In [ ]:
if __name__ == '__main__':
    train_data, test_data, class_names = filter_data()

    #Count Vectorization
    train_bow, test_bow, feature_names = bow_features(train_data, test_data)
    
    # TF-idf
    train_bow_tf_idf, test_bow_tf_idf, feature_names_tf_idf = tf_idf_features(train_data, test_data)

Training data size:: 11314
Training categories names:: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
[('alt.atheism', 480), ('comp.graphics', 584), ('comp.os.ms-windows.misc', 591), ('comp.sys.ibm.pc.hardware', 590), ('comp.sys.mac.hardware', 578), ('comp.windows.x', 593), ('misc.forsale', 585), ('rec.autos', 594), ('rec.motorcycles', 598), ('rec.sport.baseball', 597), ('rec.sport.hockey', 600), ('sci.crypt', 595), ('sci.electronics', 591), ('sci.med', 594), ('sci.space', 593), ('soc.religion.christian', 599), ('talk.politics.guns', 546), ('talk.politics.mideast', 564), ('talk.politics.misc', 465), ('talk.religion.misc', 377)]
                 

In [ ]:
#Ruksar